In [1]:
import sys
sys.path.append('../lib')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import xgboost as xgb

import lightgbm as lgb

from Ensembler import Ensembler
# from CVHelper import CVHelper

In [2]:
from sklearn.model_selection import KFold

In [3]:
munged_train_df = pd.read_csv('../data/offline/train.csv', index_col=0)
munged_test_df = pd.read_csv('../data/offline/test.csv', index_col=0)
y_train_df = pd.read_csv('../data/offline/y_train.csv', index_col=0)

X_train, X_test, y_train, y_test = train_test_split(munged_train_df.values, y_train_df['y'].values, test_size=0.5, random_state=1729)
X_all_train = munged_train_df.values
y_all_train = y_train_df['y'].values
X_all_test = munged_test_df.values

In [4]:
xgb_model = xgb.XGBRegressor(n_estimators=700, max_depth=4, learning_rate=0.005, subsample=0.92)
gbm_model = lgb.LGBMRegressor(max_depth=4, num_leaves=9, learning_rate=0.005, n_estimators=700, subsample=0.95, seed=1729)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_all_train, y_all_train, test_size=0.2, random_state=1729)

In [6]:
gbm_model.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.005,
       max_bin=255, max_depth=4, min_child_samples=10, min_child_weight=5,
       min_split_gain=0, n_estimators=700, nthread=-1, num_leaves=9,
       objective='regression', reg_alpha=0, reg_lambda=0, seed=1729,
       silent=True, subsample=0.95, subsample_for_bin=50000,
       subsample_freq=1)

In [7]:
gbm_model.predict(X_test)

array([ 111.24327215,  103.59464959,   94.46492107,   94.44820466,
         93.16809119,   92.95850898,  112.48267354,  110.80486942,
         79.41197973,   94.28770451,  111.57227681,   94.92805193,
        112.14396299,  111.94905481,  111.45034958,  112.89805245,
        114.28219913,  111.68010353,   93.86313987,  103.79779567,
         92.05245018,  111.97978553,  103.0777585 ,  107.49153476,
        111.8158799 ,  113.06615137,  111.26857087,  103.6159551 ,
        111.59300508,  111.42170232,  115.00277393,  104.7137997 ,
        103.61953068,   93.98563602,   94.01836509,   96.30844466,
         96.08307577,  111.41628125,   93.89128576,   94.04524131,
         92.91985015,   93.75523205,   93.65844464,  111.50073785,
        102.98184249,   95.08792744,  103.49861737,  115.85433439,
         92.85902923,   94.64104132,   94.62111034,  111.41415938,
         94.14719252,  118.69216653,  103.11658066,   93.20140418,
         98.62075618,   94.35433743,   94.74397649,   93.80616

In [8]:
ensembler = Ensembler(5, [gbm_model, xgb_model])
level_one_train, level_one_test = ensembler.fit_predict(X_all_train, y_all_train, X_all_test)

2
i=0
j=0
j=1
j=2
j=3
j=4
i=1
j=0
j=1
j=2
j=3
j=4


In [9]:
xgb_stacking_model = xgb.XGBRegressor(n_estimators=60, max_depth=2, learning_rate=0.1, subsample=0.7)
y_pred = ensembler.stacking(xgb_stacking_model)

In [10]:
output = pd.DataFrame({'id': munged_test_df.index, 'y': y_pred})
output.to_csv('../data/online/lightbgm-7-10-6-5_420-6-13-2.csv', index=False)

In [131]:
r2_score(y_test, y_pred)

0.47858484293991843

In [122]:
level_one_train

array([[  93.03326054,   89.82991791],
       [  92.43403738,   89.46440887],
       [ 111.51614073,  108.63038635],
       ..., 
       [ 102.92917421,  100.02684784],
       [  93.67845383,   90.55873108],
       [ 104.04511104,  101.03120422]])

In [119]:
y_train

array([  92.91,   87.94,  106.52, ...,   98.95,   91.97,  111.68])